In [1]:
import numpy as np

StZeroResult=[]
StOneResult=[]

Nums = 100000

for i in range(Nums):
    
    #랜덤하게 'O','0','X' 를 배치한다.
    #컵의 문자가 'X'면 보상 1를 받고, 'O'이면 보상 0을 얻는다.
    #일반성을 잃지않고 항상 randomCup[0]을 선택한다.
    token=['O','X','O']
    randomCup = np.random.choice(token, 3, p=[1/3,1/3,1/3],replace=False)
    
    #처음에 선택한 컵을 계속 유지한다
    if randomCup[0] == 'X':
            StZeroResult.append(1)
    else:
            StZeroResult.append(0)
    
    #최초에 선택받지 못한 2개의 컵중 '0'있는 컵을 확인 한 후
    #다른 나머지 하나의 컵으로 선택을 바꾼다.
    if randomCup[1] == 'O' and randomCup[2] == 'O' :
            StOneResult.append(0)
    else:
            StOneResult.append(1)
            
print(sum(StZeroResult)/Nums, sum(StOneResult)/Nums, len(StOneResult))

0.33155 0.66845 100000


In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np


def OpenTheCup(strategy):
    
    #랜덤하게 'O','0','X' 를 배치한다.
    #컵의 문자가 'X'면 보상 1를 받고, 'O'이면 보상 0을 얻는다.
    #일반성을 잃지않고 항상 randomCup[0]을 선택한다.
    token=['O','X','O']
    randomCup = np.random.choice(token, 3, p=[1/3,1/3,1/3],replace=False)
    
    if strategy == 0:
        
        if randomCup[0] == 'X':
            return 1
        else:
            return 0
    
    if strategy == 1:
        
        if randomCup[1] == 'O':
            if randomCup[2] == 'X':
                return 1
            else:
                return 0
            
        if randomCup[2] =='O':
            if randomCup[1] == 'X':
                return 1
            else:
                return 0


In [7]:
import numpy as np

#함수로 만듬
def OpenTheCup(strategy):
    
    token=['O','X','O']
    randomCup = np.random.choice(token, 3, p=[1/3,1/3,1/3],replace=False)
    
    if strategy == 0:
        
        if randomCup[0] == 'X':
            return 1
        else:
            return 0
    
    if strategy == 1:
        
        if randomCup[1] == 'O':
            if randomCup[2] == 'X':
                return 1
            else:
                return 0
            
        if randomCup[2] =='O':
            if randomCup[1] == 'X':
                return 1
            else:
                return 0


#메인시작

CupStrategy = [0,1]
#CupStrategy[0]:3개의 컵 중 최초에 선택한 것을 계속 유지한다.
#CupStrategy[1]:최초에 선택받지 못한 2개의 컵중 비어있는 컵을 확인 한 후
#               다른 나머지 하나의 컵으로 선택을 바꾼다.

#num_stgy=2
num_stgy = len(CupStrategy)

#보상의 총합, 최초에는 [0 0]
Reward = np.zeros([num_stgy])

#전략선택확률분포(Weight), 최초에는 [0.5 0.5]
#softmax함수로 불린다 
#= [exp(a_1)/(exp(a_1)+exp(a_2)+...+exp(a_n)), exp(a_2)/(exp(a_1)+exp(a_2)+...+exp(a_n)), ...]
#식에서 확인할 수 있듯이 각각은 양의 값이고 합은 1 이기 때문에 확률분포를 만들기 적합
Selection = np.exp(Reward)/sum(np.exp(Reward))

Nums = 10000
eplison = 0.001

for i in range(Nums):
    #업데이트된 확률분포(Weight)에 따라 전략선택
    #누적보상이 큰 전략이 높은 확률로 선택된다.
    FixSt = np.random.choice(CupStrategy, p=Selection)
    
    #보상의 크기를 조절
    Reward[FixSt] += eplison*OpenTheCup(CupStrategy[FixSt])
    
    #보상에 따라서 확률분포(Weight)를 업데이트
    Selection = np.exp(Reward)/sum(np.exp(Reward))
    
    if i % 1000 == 0 : print(Selection)
   

[0.50025 0.49975]
[0.4536336 0.5463664]
[0.40179296 0.59820704]
[0.34096428 0.65903572]
[0.26327802 0.73672198]
[0.18317244 0.81682756]
[0.11527054 0.88472946]
[0.07076481 0.92923519]
[0.03935432 0.96064568]
[0.02087719 0.97912281]


In [4]:
CupStrategy = [0,1]

#CupStrategy[0]:3개의 컵 중 최초에 선택한 것을 계속 유지한다.
#CupStrategy[1]:최초에 선택받지 못한 2개의 컵중 비어있는 컵을 확인 한 후
#               다른 나머지 하나의 컵으로 선택을 바꾼다.

#num_stgy=2
num_stgy = len(CupStrategy)

#네트워크의 피드포워드 부분을 구현한다.
weights=tf.Variable(tf.ones([num_stgy]))
output=tf.nn.softmax(weights)

#학습과정을 구현한다
reward_holder=tf.placeholder(shape=[1],dtype=tf.float32)
action_holder=tf.placeholder(shape=[1],dtype=tf.int32)

responsible_output=tf.slice(output, action_holder, [1])
loss=-(tf.log(responsible_output)*reward_holder)
optimizer=tf.train.AdamOptimizer(learning_rate=1e-3)
update=optimizer.minimize(loss)

#에이전트를 학습시킬 총 에피소드의 수를 설정한다
total_episodes = 10000
#Cup선택에 대한 보상을 0으로 설정
total_reward = np.zeros(num_stgy)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i=0
    while i<total_episodes:
        #볼츠만 분포에 따라 액션 선택
        actions=sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a)
        
        #Cup을 전략에 따라 하나 선택함으로써 보상을 받는다.
        reward = OpenTheCup(CupStrategy[action])
        
        #네트워크를 업데이트한다
        _,resp,ww=sess.run([update,responsible_output,weights],
                          feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #보상의 총계 업데이트
        total_reward[action] += reward
        if np.sum(total_reward) == 0 :
             prob_reward = total_reward
        else:
             prob_reward = total_reward/np.sum(total_reward)
                
        if i%1000 == 0:
            print("Running reward for the " + str(i) + " trial: " + "strategy " + str(action) +
                    str(actions))
            
        i+=1


Running reward for the 0 trial: strategy 0[0.5 0.5]
Running reward for the 1000 trial: strategy 1[0.38475022 0.6152498 ]
Running reward for the 2000 trial: strategy 1[0.28860947 0.71139055]
Running reward for the 3000 trial: strategy 1[0.22481883 0.7751811 ]
Running reward for the 4000 trial: strategy 1[0.16811302 0.831887  ]
Running reward for the 5000 trial: strategy 1[0.12280677 0.8771933 ]
Running reward for the 6000 trial: strategy 1[0.0892373  0.91076267]
Running reward for the 7000 trial: strategy 1[0.07205566 0.92794436]
Running reward for the 8000 trial: strategy 1[0.05577284 0.9442271 ]
Running reward for the 9000 trial: strategy 1[0.04232314 0.9576769 ]
